In [1]:
import copy
import torch
import torch.nn as nn
import os
import time
import math
import warnings
import numpy as np
import sys
import torch.backends.cudnn as cudnn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.optim import Optimizer
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset, random_split
import random
import pandas as pd

from sklearn.cluster import AgglomerativeClustering
from sklearn.preprocessing import label_binarize

In [2]:
raw_tr_data = np.load(f'./train_data_diginetica.npy', allow_pickle=True)
raw_val_data = np.load(f'./test_data_diginetica.npy', allow_pickle=True)

In [3]:
dataset = 'diginetica'
attack_type = 'B' # A1: label_poison, A2: gaussian_attack, A3: scaling_attack, A4: reverse_attack
local_learning_rate = 0.01
local_steps= 1
data_path= f"."
learning_rate_decay_gamma= 0.99
learning_rate_decay= False
future_test= False
mu= 1
global_rounds= 50
num_clients= len(raw_val_data)
join_ratio= 1.0
attack_ratio= 0.0
algorithm= "FedCHAR"
future_ratio= 0.0
finetune_rounds= 0
eval_gap= 1
detailed_info= False
partition= "nature"
initial_rounds= 10
n_clusters= 3
metric= 'cosine'
linkage= 'complete'

In [4]:
print(num_clients)

45


In [5]:
seed = 42

cudnn.benchmark = False
cudnn.deterministic = True
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

In [6]:
#parameter for recommender system
input_size = 889
hidden_size = 400
num_layers = 3
nhead = 4
output_size = input_size
batch_size = 10
K = 5

In [7]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Training on {DEVICE}")

Training on cuda:0


In [8]:
# concat all train data as one dataframe
train_combined = np.concatenate(raw_tr_data)
#convert to dataframe
train_combined = pd.DataFrame(train_combined)
train_combined.shape

(1455, 14)

In [9]:
# Step 1: Extract unique item IDs from the combined DataFrame
all_unique_items = train_combined[2].unique()

# Step 2: Create a universal item index mapping
universal_item_map = pd.DataFrame({
    'item_idx': np.arange(len(all_unique_items)),
    'itemId': all_unique_items
})

In [10]:
class GRUDataset(Dataset):
    def __init__(self, data, itemmap, session_key='sessionId', item_key='itemId', time_key='time'):
        self.data = data
        self.itemmap = itemmap
        self.session_key = session_key
        self.item_key = item_key
        self.time_key = time_key

        # Map items to indices
        self.data = pd.merge(self.data, self.itemmap, on=self.item_key, how='inner')

        # Sort by session and time
        self.data.sort_values([self.session_key, self.time_key], inplace=True)

        # Group data by session and collect item indices
        self.sessions = self.data.groupby(self.session_key)['item_idx'].apply(list)

    def __len__(self):
        return len(self.sessions)

    def __getitem__(self, index):
        session_items = self.sessions.iloc[index]
        sequence = torch.tensor(session_items[:-1], dtype=torch.long)
        target = torch.tensor(session_items[1:], dtype=torch.long)
        return sequence, target

In [11]:
def collate_fn(batch):
    sequences, targets = zip(*batch)
    sequences_padded = pad_sequence(sequences, batch_first=True, padding_value=0)
    targets_padded = pad_sequence(targets, batch_first=True, padding_value=-1)
    return sequences_padded, targets_padded

def get_loader(data, itemmap, batch_size=32, shuffle=True):
    dataset = GRUDataset(data, itemmap=itemmap)
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, collate_fn=collate_fn)


In [12]:
class PerturbedGradientDescent(Optimizer):
  def __init__(self, params, lr=0.01, mu=0.0):
    default = dict(lr=lr, mu=mu)
    super().__init__(params, default)

  @torch.no_grad()
  def step(self, global_params, device):
    for group in self.param_groups:
      for p, g in zip(group['params'], global_params):
        g = g.to(device)
        d_p = p.grad.data + group['mu'] * (p.data - g.data)
        p.data.add_(d_p, alpha=-group['lr'])

In [13]:
class TransformerModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1, nhead=4, dropout=0.1):
        """
        Initialize the Transformer model.

        Args:
            input_size (int): The number of expected features in the input `x`
            hidden_size (int): The number of features in the hidden state `h`
            output_size (int): The size of the output layer (number of items)
            num_layers (int, optional): Number of transformer layers. Default: 1
            nhead (int, optional): Number of heads in the multiheadattention models. Default: 4
            dropout (float, optional): The dropout value. Default: 0.1
        """
        super(TransformerModel, self).__init__()

        self.hidden_size = hidden_size

        self.model_type = 'Transformer'
        self.src_mask = None

        # Embedding layer
        self.embedding = nn.Embedding(input_size, hidden_size)

        # Positional Encoding
        self.pos_encoder = PositionalEncoding(hidden_size, dropout)

        # Transformer layer
        encoder_layers = nn.TransformerEncoderLayer(d_model=hidden_size, nhead=nhead, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers=num_layers)

        # Fully connected layer
        self.fc = nn.Linear(hidden_size, output_size)

    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def forward(self, x):
        if self.src_mask is None or self.src_mask.size(0) != len(x):
            device = x.device
            mask = self._generate_square_subsequent_mask(len(x)).to(device)
            self.src_mask = mask

        x = self.embedding(x) * math.sqrt(self.hidden_size)
        x = self.pos_encoder(x)
        output = self.transformer_encoder(x, self.src_mask)
        output = self.fc(output)
        return output

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

In [14]:
class Client(object):
  """
  Base class for clients in federated learning.
  """

  def __init__(self, model, id, malicious, **kwargs):
    self.model = copy.deepcopy(model)
    self.dataset = dataset
    self.device = DEVICE
    self.id = id
    self.malicious = malicious
    self.attack_type = attack_type
    self.num_classes = output_size
    self.batch_size = batch_size
    self.learning_rate = local_learning_rate
    self.local_steps = local_steps
    self.data_path = data_path
    self.learning_rate_decay = learning_rate_decay
    self.future_test = future_test


    # check BatchNorm
    self.has_BatchNorm = False
    for layer in self.model.children():
      if isinstance(layer, nn.BatchNorm2d):
        self.has_BatchNorm = True
        break

    self.loss = nn.CrossEntropyLoss(ignore_index=-1)  # Replace with your loss function
    self.optimizer = torch.optim.SGD(self.model.parameters(), lr=self.learning_rate) # momentum=0.9, weight_decay=1e-4
    self.learning_rate_scheduler = torch.optim.lr_scheduler.ExponentialLR(
      optimizer=self.optimizer,
      gamma=learning_rate_decay_gamma
    )

  def load_train_data(self, batch_size=None):
    if batch_size == None:
      batch_size = self.batch_size
    train_data = get_loader(raw_tr_data[self.id], itemmap=universal_item_map, batch_size=batch_size)

    # label poison attack
    if self.malicious and self.attack_type == 'A1':
      for idx in range(len(train_data)):
        train_data[idx][1] = self.num_classes - train_data[idx][1] - 1
    self.train_samples = len(train_data)
    return train_data

  def load_test_data(self, batch_size=None):
    """
    fine-tunes the model using the loaded training data
    """
    if batch_size == None:
      batch_size = self.batch_size
    test_data = get_loader(raw_val_data[self.id], itemmap=universal_item_map, batch_size=batch_size)
    return test_data
  
  def set_parameters(self, model):
    for new_param, old_param in zip(model.parameters(), self.model.parameters()):
      old_param.data = new_param.data.clone()

  def fine_tuning(self):
    trainloader = self.load_train_data()
    self.model.train()

    for i, (x, y) in enumerate(trainloader):
      # if type(x) == type([]):
      #   x[0] = x[0].to(self.device)
      # else:
      #   x = x.to(self.device)
      x = x.to(self.device)
      y = y.to(self.device)
      self.optimizer.zero_grad()
      output = self.model(x)
      output = output.view(-1, output.size(-1))  # Flatten output for cross-entropy
      y = y.view(-1)  # Flatten target
      # output = self.model(x)
      loss = self.loss(output, y)
      loss.backward()
      self.optimizer.step()

  def new_test_metrics(self):
    """
    evaluates the model's performance on test data, particularly its accuracy.
    """

    testloaderfull = self.load_test_data()
    self.model.eval()

    total_recall = 0.0
    total_mrr = 0.0
    test_num = 0
    y_prob = [] #model outputs or probabilities
    y_true = []

    with torch.no_grad():
      for x, y in testloaderfull:
        # if type(x) == type([]):
        #   x[0] = x[0].to(self.device)
        # else:
        #   x = x.to(self.device)
        x = x.to(self.device)
        y = y.to(self.device)
        # hidden = self.model.init_hidden(x.size(0))
        # output = self.model(x)
        output = self.model(x)
        output = output.view(-1, output.size(-1))  # Flatten output for cross-entropy

        # Select top-k items
        _, top_k_indices = torch.topk(output, K, dim=1)

        # test_acc += (torch.sum(torch.argmax(output, dim=1) == y)).item()
        # test_num += y.shape[0]

        # Calculate recall and MRR for each batch
        for i in range(x.size(0)):
          for y_item in y[i]:
            if y_item == -1:
              continue
            target_item_scalar = y_item.item()
            top_k_items = top_k_indices[i].tolist()

            # Calculate Recall@k
            if target_item_scalar in top_k_items:
              total_recall += 1

            # Calculate MRR@k
            if target_item_scalar in top_k_items:
              rank = top_k_items.index(target_item_scalar)
              total_mrr += 1 / (rank + 1)
          
          test_num += len(y[i][y[i] != -1])  # Count non-padding elements

        y_prob.append(output.detach().cpu().numpy())
        nc = self.num_classes
        if self.num_classes == 2:
          nc += 1
        lb = label_binarize(y.detach().cpu().numpy(), classes=np.arange(nc))
        if self.num_classes == 2:
          lb = lb[:, :2]
        y_true.append(lb)

    y_prob = np.concatenate(y_prob, axis=0)
    y_true = np.concatenate(y_true, axis=0)


    return total_recall, total_mrr, test_num

  def new_train_metrics(self):
    """
    evaluates the model's loss on the training data.
    """

    trainloader = self.load_train_data()
    self.model.eval()

    train_num = 0
    losses = 0.0
    with torch.no_grad():
      for x, y in trainloader:
        # if type(x) == type([]):
        #   x[0] = x[0].to(self.device)
        # else:
        #   x = x.to(self.device)
        x = x.to(self.device)
        y = y.to(self.device)
        output = self.model(x)
        output = output.view(-1, output.size(-1))  # Flatten output for cross-entropy
        y = y.view(-1)  # Flatten target
        # output = self.model(x)
        # calculate losses
        loss = self.loss(output, y)
        train_num += y.shape[0]
        losses += loss * y.shape[0]
        # loss = self.loss(output, y)
        # train_num += y.shape[0]
        # losses += loss.item() * y.shape[0]

    return losses, train_num

  def test_metrics_personalized(self):
    testloaderfull = self.load_test_data()

    self.model.eval()

    total_recall = 0.0
    total_mrr = 0.0
    test_num = 0
    # y_prob = []
    # y_true = []

    with torch.no_grad():
      for x, y in testloaderfull:
        # if type(x) == type([]):
        #   x[0] = x[0].to(self.device)
        # else:
        #   x = x.to(self.device)
        x = x.to(self.device)
        y = y.to(self.device)
        output = self.model(x)
        output = output.view(-1, output.size(-1))  # Flatten output for cross-entropy 

        # Select top-k items
        _, top_k_indices = torch.topk(output, K, dim=1)

        # test_acc += (torch.sum(torch.argmax(output, dim=1) == y)).item()
        # test_num += y.shape[0]

        # Calculate recall and MRR for each batch
        for i in range(x.size(0)):
          for y_item in y[i]:
            if y_item == -1:
              continue
            target_item_scalar = y_item.item()
            top_k_items = top_k_indices[i].tolist()

            # Calculate Recall@k
            if target_item_scalar in top_k_items:
              total_recall += 1

            # Calculate MRR@k
            if target_item_scalar in top_k_items:
              rank = top_k_items.index(target_item_scalar)
              total_mrr += 1 / (rank + 1)
          
          test_num += len(y[i][y[i] != -1]) # Count non-padding elements

        # y_prob.append(output.detach().cpu().numpy())
        # nc = self.num_classes
        # if self.num_classes == 2:
        #   nc += 1
        # lb = label_binarize(y.detach().cpu().numpy(), classes=np.arange(nc))
        # if self.num_classes == 2:
        #   lb = lb[:, :2]
        # y_true.append(lb)

    # y_prob = np.concatenate(y_prob, axis=0)
    # y_true = np.concatenate(y_true, axis=0)

    return total_recall, total_mrr, test_num

  def train_metrics_personalized(self):
    trainloader = self.load_train_data()

    self.model.eval()

    train_num = 0
    losses = 0
    with torch.no_grad():
      for x, y in trainloader:
        # if type(x) == type([]):
        #   x[0] = x[0].to(self.device)
        # else:
        #   x = x.to(self.device)
        x = x.to(self.device)
        y = y.to(self.device)
        output = self.model(x)
        output = output.view(-1, output.size(-1))  # Flatten output for cross-entropy
        y = y.view(-1)  # Flatten target
        # output = self.model(x)
        loss = self.loss(output, y)
        train_num += y.shape[0]
        losses += loss * y.shape[0]

    return losses, train_num

In [15]:
class clientCHAR(Client):
  def __init__(self, model, id, malicious, **kwargs):
    super().__init__(model, id, malicious, **kwargs)
    self.mu = mu
    self.model_per = copy.deepcopy(self.model)
    self.optimizer_per = PerturbedGradientDescent(self.model_per.parameters(), lr=self.learning_rate, mu=self.mu)
    self.learning_rate_scheduler_per = torch.optim.lr_scheduler.ExponentialLR(
        optimizer=self.optimizer_per,
        gamma=learning_rate_decay_gamma
        )

  def dtrain(self):
    trainloader = self.load_train_data()
    model = copy.deepcopy(self.model)
    self.model.train()
    self.model_per.train()

    max_local_steps = self.local_steps

    for step in range(max_local_steps):
      for x, y in trainloader:
        # if type(x) == type([]):
        #   x[0] = x[0].to(self.device)
        # else:
        #   x = x.to(self.device)
        x = x.to(self.device)
        y = y.to(self.device)
        out_p = self.model_per(x)
        out_p = out_p.view(-1, out_p.size(-1))  # Flatten output for cross-entropy
        y = y.view(-1)  # Flatten target
        # out_p = self.model_per(x)
        loss = self.loss(out_p, y)
        self.optimizer_per.zero_grad()
        loss.backward()
        self.optimizer_per.step(model.parameters(), self.device)

        out_g = self.model(x)
        out_g = out_g.view(-1, out_g.size(-1))  # Flatten output for cross-entropy
        # out_g = self.model(x)
        loss = self.loss(out_g, y)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

    if self.learning_rate_decay:
      self.learning_rate_scheduler.step()
      self.learning_rate_scheduler_per.step()

  def test_metrics_personalized(self):
    testloaderfull = self.load_test_data()
    self.model_per.eval()

    total_recall = 0.0
    total_mrr = 0.0
    test_num = 0
    # y_prob = []
    # y_true = []

    with torch.no_grad():
      for x, y in testloaderfull:
        # if type(x) == type([]):
        #   x[0] = x[0].to(self.device)
        # else:
        #   x = x.to(self.device)
        x = x.to(self.device)
        y = y.to(self.device)
        output = self.model_per(x)
        output = output.view(-1, output.size(-1))  # Flatten output for cross-entropy
        # output = self.model_per(x)

        # test_acc += (torch.sum(torch.argmax(output, dim=1) == y)).item()
        # test_num += y.shape[0]

        # Select top-k items
        _, top_k_indices = torch.topk(output, K, dim=1)

        # Calculate recall and MRR for each batch
        for i in range(x.size(0)):
          for y_item in y[i]:
            if y_item == -1:
              continue
            target_item_scalar = y_item.item()
            top_k_items = top_k_indices[i].tolist()

            # Calculate Recall@k
            if target_item_scalar in top_k_items:
              total_recall += 1

            # Calculate MRR@k
            if target_item_scalar in top_k_items:
              rank = top_k_items.index(target_item_scalar)
              total_mrr += 1 / (rank + 1)
          
          test_num += len(y[i][y[i] != -1])

    #     y_prob.append(F.softmax(output).detach().cpu().numpy())
    #     y_true.append(label_binarize(y.detach().cpu().numpy(), classes=np.arange(self.num_classes)))

    # y_prob = np.concatenate(y_prob, axis=0)
    # y_true = np.concatenate(y_true, axis=0)

    return total_recall, total_mrr, test_num

  def train_metrics_personalized(self):
    trainloader = self.load_train_data()
    self.model_per.eval()

    train_num = 0
    losses = 0
    with torch.no_grad():
      for x, y in trainloader:
        # if type(x) == type([]):
        #   x[0] = x[0].to(self.device)
        # else:
        #   x = x.to(self.device)
        x = x.to(self.device)
        y = y.to(self.device)
        output = self.model_per(x)
        output = output.view(-1, output.size(-1))  # Flatten output for cross-entropy
        y = y.view(-1) # Flatten target
        # output = self.model_per(x)
        loss = self.loss(output, y)

        #add a regularization term to the loss
        # ensure that the personalized model doesn't deviate too far from the global model.
        # The strength of this regularization is controlled by the parameter self.mu
        gm = torch.cat([p.data.view(-1) for p in self.model.parameters()], dim=0)
        pm = torch.cat([p.data.view(-1) for p in self.model_per.parameters()], dim=0)
        loss += 0.5 * self.mu * torch.norm(pm-gm, p=2) #element-wise difference using L2 norm

        train_num += y.shape[0]
        losses += loss.item() * y.shape[0]

    return losses, train_num

  def get_update(self, global_model):
    trainloader = self.load_train_data()
    model = copy.deepcopy(self.model) #old model
    self.set_parameters(global_model)
    self.model.train()

    max_local_steps = self.local_steps

    for step in range(max_local_steps):
      for i, (x, y) in enumerate(trainloader):
        # if type(x) == type([]):
        #   x[0] = x[0].to(self.device)
        # else:
        #   x = x.to(self.device)
        x = x.to(self.device)
        y = y.to(self.device)
        output = self.model(x)
        output = output.view(-1, output.size(-1))  # Flatten output for cross-entropy
        y = y.view(-1)  # Flatten target
        # output = self.model(x)
        loss = self.loss(output, y)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

    model_update = [c_param.data - s_param.data for c_param, s_param in zip(self.model.parameters(), global_model.parameters())]
    self.set_parameters(model)
    return model_update

Pay attention to the train_num and test_num

In [16]:
class Server(object):
  def __init__(self, model):
    # Set up the main attributes
    self.device = DEVICE
    self.dataset = dataset
    self.num_classes = input_size
    self.global_rounds = global_rounds
    self.local_steps = local_steps
    self.batch_size = batch_size
    self.learning_rate = local_learning_rate
    self.global_model = copy.deepcopy(model)
    self.num_clients = num_clients
    self.join_ratio = join_ratio
    self.attack_ratio = attack_ratio
    self.attack_type = attack_type
    self.seed = seed
    self.algorithm = algorithm
    self.current_round = -1
    self.future_test = future_test
    self.future_ratio = future_ratio
    self.num_training_clients = num_clients - int(num_clients * future_ratio)
    self.join_clients = int(self.num_training_clients * self.join_ratio)
    self.finetune_rounds = finetune_rounds
    self.eval_gap = eval_gap
    self.detailed_info = detailed_info
    self.partition = partition
    self.data_path = data_path

    self.clients = []
    self.training_clients = []
    self.malicious_ids = []
    self.selected_clients = []

    self.uploaded_weights = []
    self.uploaded_ids = []
    self.uploaded_models = []
    self.uploaded_updates = []

    self.rs_test_recall_g = []
    self.rs_test_mrr_g = []
    self.rs_train_loss_g = []
    self.rs_test_recalls_g = []
    self.rs_test_mrrs_g = []
    self.rs_test_recall_p = []
    self.rs_test_mrr_p = []
    self.rs_train_loss_p = []
    self.rs_test_recalls_p = []
    self.rs_test_mrrs_p = []
    self.ft_train_loss = []
    self.ft_test_recall = []
    self.ft_std_recall = []
    self.ft_test_mrr = []
    self.ft_std_mrr = []

  def set_clients(self, model, clientObj):

    if self.future_test == False:
      if self.attack_type == 'B':
        self.malicious_ids = []
        self.attack_ratio = 0.0
      else:
        self.malicious_ids = np.sort(np.random.choice(np.arange(self.num_clients), int(self.num_clients * self.attack_ratio), replace=False))


      for i in range(self.num_clients):
        client = clientObj(model=model, id=i,
                        malicious=True if i in self.malicious_ids else False)
        self.clients.append(client)

      self.training_clients = self.clients
      self.training_clients_ids = np.arange(self.num_clients)

    else:
      if self.algorithm != 'FedCHAR_DC':
        print('{} do not support future testing'.format(self.algorithm))
        raise NotImplementedError

      self.training_clients_ids = np.sort(np.random.choice(np.arange(self.num_clients), self.num_training_clients, replace=False))

      if self.attack_type == 'B':
        self.malicious_ids = []
        self.attack_ratio = 0.0
      else:
        self.malicious_ids = np.sort(np.random.choice(self.training_clients_ids, int(self.num_training_clients * self.attack_ratio),
                                                      replace=False))

      for i in range(self.num_clients):
        client = clientObj(model=model, id=i,
                        malicious=True if i in self.malicious_ids else False)
        self.clients.append(client)

        if i in self.training_clients_ids:
          self.training_clients.append(client)

    print('Malicious Clients: {}'.format(list(self.malicious_ids)))
    print('Future Clients: {}'.format(list(np.sort(np.setdiff1d(np.arange(self.num_clients), self.training_clients_ids)))))

  def select_clients(self):
    selected_clients = list(np.random.choice(self.training_clients, self.join_clients, replace=False))
    return selected_clients

  def send_models(self):
    for client in self.selected_clients:
      client.set_parameters(self.global_model)

  def send_models_to_future_clients(self):
    for client in self.selected_clients:
      client.set_parameters(self.global_model)

  def receive_models(self):
    self.uploaded_ids = []
    self.uploaded_weights = [] #weight based on the fraction of client's data
    self.uploaded_models = []

    tot_samples = 0
    for client in self.selected_clients:
      tot_samples += client.train_samples
      self.uploaded_ids.append(client.id)
      self.uploaded_weights.append(client.train_samples)
      self.uploaded_models.append(client.model)

    for i, w in enumerate(self.uploaded_weights):
      self.uploaded_weights[i] = w / tot_samples

  def load_model(self):
    model_path = os.path.join(f"./models", self.dataset)
    model_path = os.path.join(model_path, self.algorithm + "_server" + ".pt")
    assert (os.path.exists(model_path))
    self.global_model = torch.load(model_path)

  def model_exists(self):
    model_path = os.path.join(f"./models", self.dataset)
    model_path = os.path.join(model_path, self.algorithm + "_server" + ".pt")
    return os.path.exists(model_path)

  def save_results(self):
    filename = "{}_{}_{}_{}_{}_bz{}_lr{}_gr{}_ep{}_jr{}_nc{}_fur{}_ntc{}_ftr{}_seed{}".format(self.dataset, self.partition, self.algorithm,
                                                                                        self.attack_type, self.attack_ratio, self.batch_size,
                                                                                        self.learning_rate, self.global_rounds, self.local_steps,
                                                                                        self.join_ratio, self.num_clients, self.future_ratio,
                                                                                        self.num_training_clients, self.finetune_rounds,
                                                                                        self.seed)

    if self.algorithm == 'FedCHAR':
      filename = filename + '_ir{}_ng{}_mtrc{}_lkg{}'.format(self.initial_rounds, self.n_clusters, self.metric, self.linkage)

    elif self.algorithm == 'FedCHAR_DC':
      filename = filename + '_ir{}_ng{}_mtrc{}_lkg{}_rr{}'.format(self.initial_rounds, self.n_clusters, self.metric, self.linkage,
                                                                  self.recluster_rounds)

    result_path = f"./results/npz/"
    if not os.path.exists(result_path):
      os.makedirs(result_path)

    if len(self.rs_test_acc_g) or len(self.rs_test_acc_p):
      file_path = result_path + "{}.npz".format(filename)
      print("Result path: " + file_path)

      np.savez(file_path, test_acc_g=self.rs_test_acc_g,
              test_acc_p=self.rs_test_acc_p, test_accs_g=self.rs_test_accs_g,
              test_accs_p=self.rs_test_accs_p, train_loss_g=self.rs_train_loss_g,
              train_loss_p=self.rs_train_loss_p, ft_train_loss=self.ft_train_loss,
              ft_test_acc=self.ft_test_acc, ft_std_acc=self.ft_std_acc)

  # did not implemented the modification
  def test_metrics_for_future_clients(self):
    num_samples = []
    tot_correct = []

    for c in self.selected_clients:
      ct, ns = c.new_test_metrics()
      tot_correct.append(ct*1.0)
      num_samples.append(ns)

    ids = [c.id for c in self.selected_clients]
    return ids, num_samples, tot_correct

  # did not implemented the modification
  def train_metrics_for_future_clients(self):
    num_samples = []
    losses = []
    for c in self.selected_clients:
      cl, ns = c.new_train_metrics()
      num_samples.append(ns)
      losses.append(cl*1.0)

    ids = [c.id for c in self.selected_clients]
    return ids, num_samples, losses

  def evaluate_personalized(self, rec=None, loss=None, mrr=None):
    stats = self.test_metrics_personalized()
    stats_train = self.train_metrics_personalized()

    if self.malicious_ids != []: # skip this for now
      relative_malicious_ids = np.array([stats[0].index(i) for i in self.malicious_ids])

      stats_A = np.array(stats)[:, relative_malicious_ids].tolist()
      stats_train_A = np.array(stats_train)[:, relative_malicious_ids].tolist()

      test_acc_A = sum(stats_A[2])*1.0 / sum(stats_A[1])
      train_loss_A = sum(stats_train_A[2])*1.0 / sum(stats_train_A[1])
      accs_A = [a / n for a, n in zip(stats_A[2], stats_A[1])]
      losses_A = [a / n for a, n in zip(stats_train_A[2], stats_train_A[1])]

    else:
      test_acc_A = -1
      train_loss_A = -1
      accs_A = []
      losses_A = []

    benign_ids = np.sort(np.setdiff1d(self.training_clients_ids, self.malicious_ids))
    relative_benign_ids = np.array([stats[0].index(i) for i in benign_ids])

    stats_B = np.array(stats)[:, relative_benign_ids].tolist()
    stats_train_B = np.array(stats_train)[:, relative_benign_ids].tolist()

    stats = None
    stats_train = None

    # test_acc = sum(stats_B[2])*1.0 / sum(stats_B[1])
    # train_loss = sum(stats_train_B[2])*1.0 / sum(stats_train_B[1])
    # accs = [a / n for a, n in zip(stats_B[2], stats_B[1])]
    # losses = [a / n for a, n in zip(stats_train_B[2], stats_train_B[1])]

    test_recall = sum(stats_B[2])*1.0 / sum(stats_B[1])
    test_mrr = sum(stats_B[3])*1.0 / sum(stats_B[1])
    train_loss = sum(stats_train_B[2])*1.0 / sum(stats_train_B[1])
    recalls = [a / n for a, n in zip(stats_B[2], stats_B[1])]
    mrrs = [a / n for a, n in zip(stats_B[3], stats_B[1])]
    losses = [a / n for a, n in zip(stats_train_B[2], stats_train_B[1])]

    if rec == None:
      self.rs_test_recall_p.append(test_recall)
    else:
      rec.append(test_recall)

    if mrr == None:
      self.rs_test_mrr_p.append(test_mrr)
    else:
      mrr.append(test_mrr)

    if loss == None:
      self.rs_train_loss_p.append(train_loss)
    else:
      loss.append(train_loss)

    self.rs_test_recall_p.append(recalls)
    self.rs_test_mrr_p.append(mrrs)

    print("Benign Averaged Train Loss: {:.2f}".format(train_loss))
    # print("Benign Averaged Test Accurancy: {:.2f}%".format(test_acc*100))
    # print("Benign Std Test Accurancy: {:.2f}%".format(np.std(accs)*100))
    print("Benign Averaged Test Recall: {:.2f}%".format(test_recall*100))
    print("Benign Std Test Recall: {:.2f}%".format(np.std(recalls)*100))
    print("Benign Averaged Test MRR: {:.2f}%".format(test_mrr*100))
    print("Benign Std Test MRR: {:.2f}%".format(np.std(mrrs)*100))

    if self.malicious_ids != []:
      print("Malicious Averaged Train Loss: {:.2f}".format(train_loss_A))
      print("Malicious Averaged Test Accurancy: {:.2f}%".format(test_acc_A*100))

  # did not implemented the modification
  def evaluate_for_future_clients(self):
    stats = self.test_metrics_for_future_clients()
    stats_train = self.train_metrics_for_future_clients()
    stats = np.array(stats).tolist()
    stats_train = np.array(stats_train).tolist()
    test_acc = sum(stats[2])*1.0 / sum(stats[1])
    train_loss = sum(stats_train[2])*1.0 / sum(stats_train[1])
    accs = [a / n for a, n in zip(stats[2], stats[1])]
    losses = [a / n for a, n in zip(stats_train[2], stats_train[1])]

    print("Averaged Future Train Loss: {:.2f}".format(train_loss))
    print("Averaged Future Test Accurancy: {:.2f}%".format(test_acc*100))
    print("Std Future Test Accurancy: {:.2f}%".format(np.std(accs)*100))

    if self.detailed_info:
      print('Future Clients Train Loss:\n', [(int(stats[0][idx]), format(loss, '.2f')) for idx, loss in enumerate(losses)])
      print('Future Clients Test Accuracy:\n', [(int(stats[0][idx]), format(acc*100, '.2f')+'%') for idx, acc in enumerate(accs)])

    self.ft_train_loss.append(train_loss)
    self.ft_test_acc.append(test_acc)
    self.ft_std_acc.append(np.std(accs))

  def test_metrics_personalized(self):
    num_samples = []
    tot_recall = []
    tot_mrr = []

    for c in self.training_clients:
      rc, mrr, ns = c.test_metrics_personalized()
      tot_recall.append(rc)
      tot_mrr.append(mrr)
      num_samples.append(ns)

    ids = [c.id for c in self.training_clients]
    return ids, num_samples, tot_recall, tot_mrr

  def train_metrics_personalized(self):
    num_samples = []
    losses = []
    for c in self.training_clients:
      cl, ns = c.train_metrics_personalized()
      num_samples.append(ns)
      losses.append(cl*1.0)

    ids = [c.id for c in self.training_clients]
    return ids, num_samples, losses

In [17]:
class FedCHAR(Server):
  def __init__(self, model):
    super().__init__(model)

    self.set_clients(model, clientCHAR)

    print(f"\nJoin ratio / total clients: {self.join_ratio} / {self.num_training_clients}")
    print("Finished creating server and clients.")

    self.initial_rounds = initial_rounds
    self.n_clusters = n_clusters
    self.metric = metric
    self.linkage = linkage

  def train(self):
    # initial Stage
    for i in range(self.initial_rounds):
      self.selected_clients = self.select_clients()
      self.send_models()

      for client in self.selected_clients:
        client.dtrain()

      if i%self.eval_gap == 0:
        print(f"\n-------------Round number: {i}-------------")
        print("\nEvaluate personalized models for training clients.")
        self.evaluate_personalized()

      self.receive_models()
      self.aggregate_parameters()

    # Clustering Stage
    print(f"\n-------------Clustering-------------")
    clients_updates = self.collect()
    self.cluster_identity = self.cluster(clients_updates)
    cluster_info = [[('Malicious' if self.training_clients[idx].malicious else 'Benign', idx) for idx, g_id in enumerate(self.cluster_identity) if g_id == i] for i in range(max(self.cluster_identity)+1)]
    for idx, info in enumerate(cluster_info):
      print('Cluster {}: {}'.format(idx, info))

    self.group_models = [copy.deepcopy(self.global_model)] * (max(self.cluster_identity) + 1)

    # Remaining Stage
    for i in range(self.global_rounds - self.initial_rounds):
      self.selected_clients = self.select_clients()
      self.send_models_g()

      for client in self.selected_clients:
        client.dtrain()

      if i%self.eval_gap == 0:
        print(f"\n-------------Round number: {i+self.initial_rounds}-------------")
        print("\nEvaluate personalized models for training clients.")
        self.evaluate_personalized()

      self.receive_models_g()
      self.aggregate_parameters_g()

    print("\nFinal Average Personalized Recall: {}\n".format(self.rs_test_recall_p[-1]))
    print(f"Average Recall for All Users: {np.mean(self.rs_test_recall_p[-1])}")
    print("\nFinal Average Personalized Recall: {}\n".format(self.rs_test_mrr_p[-1]))
    print(f"Average MRR for All Users: {np.mean(self.rs_test_mrr_p[-1])}")

  def receive_models(self):
    self.uploaded_ids = []
    self.uploaded_weights = []
    self.uploaded_updates = []

    tot_samples = 0
    for client in self.selected_clients:
      tot_samples += client.train_samples
      self.uploaded_ids.append(client.id)
      self.uploaded_weights.append(client.train_samples)
      self.uploaded_updates.append([c_param.data - s_param.data for c_param, s_param in zip(client.model.parameters(), self.global_model.parameters())])

    if self.attack_type != 'B' and self.attack_type != 'A1':
      malicious_ids = [idx for idx, c_id in enumerate(self.uploaded_ids) if c_id in self.malicious_ids]
      self.uploaded_updates = eval(self.attack_type)(self.uploaded_updates, malicious_ids)

    for i, w in enumerate(self.uploaded_weights):
      self.uploaded_weights[i] = w / tot_samples

  def add_parameters(self, w, client_update):
    for server_param, client_param in zip(self.global_update, client_update):
      server_param.data += client_param.data.clone() * w

  def aggregate_parameters(self):
    self.global_update = copy.deepcopy(self.uploaded_updates[0])
    for param in self.global_update:
      param.data.zero_()

    for w, client_update in zip(self.uploaded_weights, self.uploaded_updates):
      self.add_parameters(w, client_update)

    for model_param, update_param in zip(self.global_model.parameters(), self.global_update):
      model_param.data += update_param.data.clone()

  def collect(self):
    clients_updates = []
    for client in self.training_clients:
      clients_updates.append(client.get_update(self.global_model))

    if self.attack_type != 'B' and self.attack_type != 'A1':
      malicious_ids = [idx for idx, c_id in enumerate(self.training_clients_ids) if c_id in self.malicious_ids]
      clients_updates = eval(self.attack_type)(clients_updates, malicious_ids, len(self.selected_clients))

    clients_updates = [torch.cat([uu.reshape(-1, 1) for uu in u], axis=0).detach().cpu().numpy().squeeze() for u in clients_updates]
    return clients_updates

  def cluster(self, clients_updates):
    clustering = AgglomerativeClustering(n_clusters=self.n_clusters, metric=self.metric, linkage=self.linkage).fit(clients_updates)
    return clustering.labels_

  def send_models_g(self):
    for client in self.selected_clients:
      c_idx = list(self.training_clients_ids).index(client.id)
      client.set_parameters(self.group_models[self.cluster_identity[c_idx]])

  def receive_models_g(self):
    self.uploaded_ids = []
    self.uploaded_weights = []
    self.uploaded_updates = []

    for client in self.selected_clients:
      self.uploaded_ids.append(client.id)
      self.uploaded_weights.append(client.train_samples)
      c_idx = list(self.training_clients_ids).index(client.id)
      self.uploaded_updates.append([c_param.data - s_param.data for c_param, s_param in zip(client.model.parameters(), self.group_models[self.cluster_identity[c_idx]].parameters())])

    if self.attack_type != 'B' and self.attack_type != 'A1':
      malicious_ids = [idx for idx, c_id in enumerate(self.uploaded_ids) if c_id in self.malicious_ids]
      self.uploaded_updates = eval(self.attack_type)(self.uploaded_updates, malicious_ids)

  def aggregate_parameters_g(self):
    for i in range(len(self.group_models)):
      self.global_update = copy.deepcopy(self.uploaded_updates[0])
      for param in self.global_update:
        param.data.zero_()

      user_idx_in_same_group = np.array([r_id for r_id, c_id in enumerate(self.uploaded_ids) if self.cluster_identity[list(self.training_clients_ids).index(c_id)] == i])
      uploaded_weights = [self.uploaded_weights[u_id] for u_id in range(len(self.uploaded_weights)) if u_id in user_idx_in_same_group]
      uploaded_weights = [weight / sum(uploaded_weights) for weight in uploaded_weights]
      uploaded_updates = [self.uploaded_updates[u_id] for u_id in range(len(self.uploaded_updates)) if u_id in user_idx_in_same_group]

      for w, client_update in zip(uploaded_weights, uploaded_updates):
        self.add_parameters(w, client_update)

      for model_param, update_param in zip(self.group_models[i].parameters(), self.global_update):
        model_param.data += update_param.data.clone()

In [18]:
warnings.simplefilter("ignore")
print("Creating server and clients ...")
start = time.time()
# model = HARCNN(in_channels=3, num_classes=num_classes, dim=3008).to(device)
model = TransformerModel(input_size=input_size, hidden_size=hidden_size, output_size=output_size, num_layers=num_layers, nhead=nhead, dropout=0.0).to(DEVICE)

print(model)

server = FedCHAR(model)
server.train()
# server.save_results()
print(f"\nTime cost: {round((time.time()-start)/60, 2)}min.")

Creating server and clients ...
TransformerModel(
  (embedding): Embedding(889, 400)
  (pos_encoder): PositionalEncoding(
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-2): 3 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=400, out_features=400, bias=True)
        )
        (linear1): Linear(in_features=400, out_features=2048, bias=True)
        (dropout): Dropout(p=0.0, inplace=False)
        (linear2): Linear(in_features=2048, out_features=400, bias=True)
        (norm1): LayerNorm((400,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((400,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.0, inplace=False)
        (dropout2): Dropout(p=0.0, inplace=False)
      )
    )
  )
  (fc): Linear(in_features=400, out_features=889, bias=True)
)
Malicious Clients: []
Future Clients: []

Join 